In [3]:
pip install opencv-python


   ---------------------------------------- 0.0/38.8 MB ? eta -:--:--
   ---------------------------------------- 0.3/38.8 MB ? eta -:--:--
   - -------------------------------------- 1.0/38.8 MB 3.1 MB/s eta 0:00:13
   - -------------------------------------- 1.8/38.8 MB 3.6 MB/s eta 0:00:11
   -- ------------------------------------- 2.9/38.8 MB 3.8 MB/s eta 0:00:10
   --- ------------------------------------ 3.4/38.8 MB 3.7 MB/s eta 0:00:10
   ---- ----------------------------------- 4.5/38.8 MB 3.8 MB/s eta 0:00:10
   ----- ---------------------------------- 5.5/38.8 MB 3.9 MB/s eta 0:00:09
   ------ --------------------------------- 6.0/38.8 MB 3.9 MB/s eta 0:00:09
   ------- -------------------------------- 6.8/38.8 MB 3.9 MB/s eta 0:00:09
   ------- -------------------------------- 7.3/38.8 MB 3.7 MB/s eta 0:00:09
   -------- ------------------------------- 8.7/38.8 MB 3.8 MB/s eta 0:00:08
   --------- ------------------------------ 9.2/38.8 MB 3.7 MB/s eta 0:00:09
   ----------

In [5]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
import numpy as np
import os
from tensorflow.keras.utils import to_categorical
import cv2

In [9]:
data_dir = "C:/Users/Manu/OneDrive/Documents/DiseasePrediction/dataset"

In [14]:
# Data Preparation
categories = ["Glaucoma", "Cataract", "Diabetic_Retinopathy", "Normal"]
img_size = 224

def load_data():
    images = []
    labels = []
    for category in categories:
        folder = os.path.join(data_dir, category)
        label = categories.index(category)
        for img in os.listdir(folder):
            img_path = os.path.join(folder, img)
            try:
                img_array = cv2.imread(img_path)
                img_array = cv2.resize(img_array, (img_size, img_size))
                images.append(img_array)
                labels.append(label)
            except Exception as e:
                print(e)
    return np.array(images), np.array(labels)

X, y = load_data()

In [16]:
# Normalize and one-hot encode labels
X = X / 255.0
y = to_categorical(y, num_classes=len(categories))

# Split Data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [18]:
# Model Building
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(img_size, img_size, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(len(categories), activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


C:\Users\Manu\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [20]:
# Train Model
history = model.fit(X_train, y_train, epochs=1, validation_data=(X_test, y_test), batch_size=32)


106/106 ━━━━━━━━━━━━━━━━━━━━ 235s 2s/step - accuracy: 0.5084 - loss: 1.3669 - val_accuracy: 0.6588 - val_loss: 0.7874


In [22]:
# Save Model
model.save("eye_disease_model.h5")